In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score


# Load the dataset psOCVclean2.csv
%ntbl pull datasets "LargeDataset2/psOCVclean2.csv"
df = pd.read_csv( "../datasets/LargeDataset2/psOCVclean2.csv", sep=';', quotechar='"')

# Remove quotes from column headers
df.columns = df.columns.str.replace('"', '')

# Display the column headers
column_headers = df.columns
print(column_headers)

In [ ]:
plt.plot(df['Current A'])
plt.xlabel("Time")
plt.ylabel("Current (A)")
plt.title("Current vs. Time")
plt.show()

plt.plot(df['Voltage V'])
plt.xlabel("Time")
plt.ylabel("Voltage (V)")
plt.title("Voltage vs. Time")
plt.show()

plt.plot(df['Step'])
plt.xlabel("Time")
plt.ylabel("Step#")
plt.title("Step vs. Time")
plt.show()

In [ ]:
# Separate charge and discharge cycles
charge = df[df['Step'] == 10]
discharge = df[df['Step'] == 14]

# Step 1: Transform the capacity to absolute values, having the 0 value at its lowest, and the maximum at the maximum voltage.
# Offset the negative capacity in the 'discharge' dataframe by its absolute minimum, forcing the minimum to be zero.
discharge['Amp-Hours AH'] = discharge['Amp-Hours AH'] - discharge['Amp-Hours AH'].min()
charge['Amp-Hours AH'] = charge['Amp-Hours AH'] - charge['Amp-Hours AH'].min()

# Step 2: Sort the 'charge' and 'discharge' dataframes from the lowest voltage to the higherst for a proper interpolation, functions with monotonically increasing intervals of X
charge = charge.sort_values(by='Amp-Hours AH', ascending=True)
discharge = discharge.sort_values(by='Amp-Hours AH', ascending=True)

# Reset index for both dataframes
charge.reset_index(drop=True, inplace=True)
discharge.reset_index(drop=True, inplace=True)


# Display the first few rows of the charge and discharge dataframes
charge.head(), discharge.tail()

In [ ]:
# Check the lengths of the charge and discharge dataframes
len(charge), len(discharge)

In [ ]:
# Interpolate the data to make them align
common_capacity = np.linspace(start=0, stop=max(max(charge['Amp-Hours AH']), max(discharge['Amp-Hours AH'])), num=max(len(charge), len(discharge)))
charge_interpolated = np.interp(common_capacity, charge['Amp-Hours AH'], charge['Voltage V'])
discharge_interpolated = np.interp(common_capacity, discharge['Amp-Hours AH'], discharge['Voltage V'])
commonCap = pd.DataFrame(common_capacity, columns = ['Amp-Hours AH'])

# Check the lengths of the charge and discharge dataframes
len(charge_interpolated), len(discharge_interpolated)
# Display the first few values of the interpolated data
charge_interpolated[:5], discharge_interpolated[:5]

voltageGap = pd.DataFrame(charge_interpolated-discharge_interpolated, columns = ['Voltage V'])
# Using DataFrame.merge() to add columns
EnergyGap = voltageGap.merge(commonCap, left_index=True, right_index=True)
print('\nResult Index Merge:\n', EnergyGap.head())

In [ ]:
# Plot Voltage gap vs common capacity
plt.figure(figsize=(10,6))
plt.plot(common_capacity, charge_interpolated)
plt.plot(common_capacity, discharge_interpolated)
plt.xlabel('Capacity (Amp-Hours)')
plt.ylabel('Voltages (V)')
plt.title('Voltage gap vs Capacity')
plt.grid(True)
plt.show()

In [ ]:
# Smoothen curves for display
window_size = 9911  # Adjust the window size as needed
poly_order = 3    # Adjust the polynomial order as needed
#delta=17,

# Apply the Savitzky-Golay filter
CHG = savgol_filter(charge_interpolated, window_size, poly_order, deriv=0, mode = 'mirror')
DCH = savgol_filter(discharge_interpolated, window_size, poly_order, deriv=0, mode = 'mirror')

In [ ]:
# Plot Voltage gap vs common capacity
plt.figure(figsize=(10,6))
#plt.plot(common_capacity, charge_interpolated, label='charge raw')
plt.plot(common_capacity, CHG, label='CHG')
#plt.plot(common_capacity, discharge_interpolated, label='discharge raw')
plt.plot(common_capacity, DCH, label='DCH')
plt.xlabel('Capacity (Amp-Hours)')
plt.ylabel('Voltages (V)')
plt.title('Voltage gap vs Capacity')
plt.grid(True)
plt.show()

In [ ]:
# Compute the hysteresis
hysteresis = np.trapz(np.abs(CHG-DCH), common_capacity)

# Display the hysteresis
hysteresis

In [ ]:
# Computing approach to dV, dQ behaviors
dVCdQ = np.gradient(CHG, common_capacity)
dVDdQ = np.gradient(DCH, common_capacity)
dQdVC = np.gradient(common_capacity, CHG)
dQdVD = np.gradient(common_capacity, DCH)
dVGdQ = np.gradient((CHG-DCH), common_capacity)

In [ ]:
# Apply the Savitzky-Golay filter
sdVCdQ = savgol_filter(dVCdQ, window_size, poly_order, deriv=0, mode = 'mirror')
sdVDdQ = savgol_filter(dVDdQ, window_size, poly_order, deriv=0, mode = 'mirror')
#sdVCdQ = savgol_filter(charge_interpolated, window_size, poly_order, deriv=1, delta=17.0 mode = 'mirror')
sdQdVC = savgol_filter(dQdVC, window_size, poly_order, deriv=0, mode = 'mirror')
sdQdVD = savgol_filter(dQdVD, window_size, poly_order, deriv=0, mode = 'mirror')
sdVGdQ = savgol_filter(dVGdQ, window_size, poly_order, deriv=0, mode = 'mirror')

In [ ]:
plt.figure(figsize=(10,6))
#create a line plot with common_capacity on the X-axis and dV/dQ on the Y-axis.
plt.plot(common_capacity, sdVCdQ, alpha=0.5, label='charge')
plt.plot(common_capacity, sdVDdQ, alpha=0.5, label='discharge')
ax = plt.gca()
#ax.set_xlim([xmin, xmax])
ax.set_ylim([-.02, .1])
plt.xlabel('common_capacity')
plt.ylabel('dV/dQ')
plt.title('dV/dQ vs common_capacity')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(charge_interpolated, sdQdVC, alpha=0.5, label='charge')
plt.plot(discharge_interpolated, sdQdVD, alpha=0.5, label='discharge')
ax = plt.gca()
#ax.set_xlim([xmin, xmax])
ax.set_ylim([-50, 250])
plt.xlabel('charge_interpolated / discharge_interpolated')
plt.ylabel('dQ/dV')
plt.title('dQ/dV for charge and discharge')
plt.legend()
plt.show()

In [ ]:
# This is a mere exercise as this curve sums up all the phenomena both during charge and discharge, so it may induce to assume too many transformations which in fact, are not, but half of it

sVGAP = savgol_filter(EnergyGap['Voltage V'], window_size, poly_order, deriv=0, mode = 'mirror')
# Plot Voltage gap vs common capacity
plt.figure(figsize=(10,6))
plt.plot(EnergyGap['Amp-Hours AH'], sVGAP)
plt.xlabel('Capacity (Amp-Hours)')
plt.ylabel('Voltage gap (V)')
plt.title('Voltage gap C20-C20 vs Capacity')
plt.grid(True)
plt.show()

plt.figure(figsize=(10,6))
#create a line plot with common_capacity on the X-axis and dV/dQ on the Y-axis.
plt.plot(common_capacity, sdVGdQ, label='Vgap')
ax = plt.gca()
#ax.set_xlim([xmin, xmax])
ax.set_ylim([-.015, .005])
plt.xlabel('common_capacity')
plt.ylabel('dVG/dQ')
plt.title('dVGap/dQ vs common_capacity')
plt.show()

In [ ]:
# Calculate the average between CHG and DCH numpy arrays
pOCV = (CHG + DCH) / 2

# Plot voltages vs common capacity
plt.figure(figsize=(10,6))
plt.plot(common_capacity, charge_interpolated, label='charge')
plt.plot(common_capacity, pOCV, label='pOCV', color='green')
plt.plot(common_capacity, discharge_interpolated, label='discharge')

plt.xlabel('Capacity (Amp-Hours)')
plt.ylabel('Voltages (V)')
plt.title('Voltage gap vs Capacity')
plt.grid(True)
plt.show()


In [ ]:
# Computing approach to dV, dQ 
dOCVdQ = np.gradient(pOCV, common_capacity)
# Apply the Savitzky-Golay filter
sdOCVdQ = savgol_filter(dOCVdQ, window_size, poly_order, deriv=0, mode = 'mirror')


fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(common_capacity, pOCV, 'g-')
ax2.plot(common_capacity, dOCVdQ, 'b-')

ax1.set_xlabel('Capacity Ah')
ax1.set_ylabel('Voltage V', color='g')
ax2.set_ylabel('dOCV/dQ', color='b')
ax2.set_ylim(bottom=0, top=0.05)

plt.show()

In [ ]:
# Define the degree of the polynomial regression model
degree = 7
# Create a pipeline model
model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
# Fit the model
model.fit(common_capacity.reshape(-1, 1), pOCV)
# Predict the pOCV
pOCV_pred = model.predict(common_capacity.reshape(-1, 1))
# Calculate the R-squared value
r2 = r2_score(pOCV, pOCV_pred)
# Print the R-squared value
print('R-squared:', r2)

# Display the polynomial coefficients
print('Polynomial Coefficients:', model.named_steps['linearregression'].coef_)

# Calculate residuals
residuals = pOCV - pOCV_pred
# Plot residuals
plt.figure(figsize=(10, 6))
plt.scatter(common_capacity, residuals, color='blue', s=50, alpha=.005)
plt.hlines(y=0, xmin=np.min(common_capacity), xmax=np.max(common_capacity), color='black')
plt.ylabel('Residuals')
plt.xlabel('Common Capacity')
plt.title('Residuals vs Common Capacity')
plt.show()